In [1]:
! ls \data

data_description.txt  test.csv
sample_submission.csv train.csv


In [2]:
import pandas as pd
from scipy import stats
import numpy as np
import sys
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from math import sqrt
np.set_printoptions(threshold=sys.maxsize)

In [3]:
# load data
df_train = pd.read_csv('data/train.csv')
df_test = pd.read_csv('data/test.csv')

In [4]:
# concatenate dataframes
df_test['SalePrice'] = -1
df = pd.concat([df_train, df_test])

In [5]:
df

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
5,6,50,RL,85.0,14115,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,MnPrv,Shed,700,10,2009,WD,Normal,143000
6,7,20,RL,75.0,10084,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,307000
7,8,60,RL,NaN,10382,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,Shed,350,11,2009,WD,Normal,200000
8,9,50,RM,51.0,6120,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2008,WD,Abnorml,129900
9,10,190,RL,50.0,7420,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,1,2008,WD,Normal,118000


In [6]:
# start preparing the training data
# check which columns contain nans
df.columns[df.isna().any()].tolist()

['MSZoning',
 'LotFrontage',
 'Alley',
 'Utilities',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'MasVnrArea',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinSF1',
 'BsmtFinType2',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 'Electrical',
 'BsmtFullBath',
 'BsmtHalfBath',
 'KitchenQual',
 'Functional',
 'FireplaceQu',
 'GarageType',
 'GarageYrBlt',
 'GarageFinish',
 'GarageCars',
 'GarageArea',
 'GarageQual',
 'GarageCond',
 'PoolQC',
 'Fence',
 'MiscFeature',
 'SaleType']

In [7]:
# check number of nans per column
nan_sum = df.isna().sum()
nan_sum[0:41]

Id                 0
MSSubClass         0
MSZoning           4
LotFrontage      486
LotArea            0
Street             0
Alley           2721
LotShape           0
LandContour        0
Utilities          2
LotConfig          0
LandSlope          0
Neighborhood       0
Condition1         0
Condition2         0
BldgType           0
HouseStyle         0
OverallQual        0
OverallCond        0
YearBuilt          0
YearRemodAdd       0
RoofStyle          0
RoofMatl           0
Exterior1st        1
Exterior2nd        1
MasVnrType        24
MasVnrArea        23
ExterQual          0
ExterCond          0
Foundation         0
BsmtQual          81
BsmtCond          82
BsmtExposure      82
BsmtFinType1      79
BsmtFinSF1         1
BsmtFinType2      80
BsmtFinSF2         1
BsmtUnfSF          1
TotalBsmtSF        1
Heating            0
HeatingQC          0
dtype: int64

In [8]:
nan_sum[40:]

HeatingQC           0
CentralAir          0
Electrical          1
1stFlrSF            0
2ndFlrSF            0
LowQualFinSF        0
GrLivArea           0
BsmtFullBath        2
BsmtHalfBath        2
FullBath            0
HalfBath            0
BedroomAbvGr        0
KitchenAbvGr        0
KitchenQual         1
TotRmsAbvGrd        0
Functional          2
Fireplaces          0
FireplaceQu      1420
GarageType        157
GarageYrBlt       159
GarageFinish      159
GarageCars          1
GarageArea          1
GarageQual        159
GarageCond        159
PavedDrive          0
WoodDeckSF          0
OpenPorchSF         0
EnclosedPorch       0
3SsnPorch           0
ScreenPorch         0
PoolArea            0
PoolQC           2909
Fence            2348
MiscFeature      2814
MiscVal             0
MoSold              0
YrSold              0
SaleType            1
SaleCondition       0
SalePrice           0
dtype: int64

In [9]:
# drop columns containing a large number of nans
df_dropcol = df.drop(['Alley', 'LotFrontage', 'FireplaceQu','PoolQC','Fence','MiscFeature'], axis = 1)

In [10]:
df_dropcol

,Id,MSSubClass,MSZoning,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,2,2008,WD,Normal,208500
1,2,20,RL,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,5,2007,WD,Normal,181500
2,3,60,RL,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,9,2008,WD,Normal,223500
3,4,70,RL,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,...,272,0,0,0,0,2,2006,WD,Abnorml,140000
4,5,60,RL,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,12,2008,WD,Normal,250000
5,6,50,RL,14115,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,0,320,0,0,700,10,2009,WD,Normal,143000
6,7,20,RL,10084,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,8,2007,WD,Normal,307000
7,8,60,RL,10382,Pave,IR1,Lvl,AllPub,Corner,Gtl,...,228,0,0,0,350,11,2009,WD,Normal,200000
8,9,50,RM,6120,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,205,0,0,0,0,4,2008,WD,Abnorml,129900
9,10,190,RL,7420,Pave,Reg,Lvl,AllPub,Corner,Gtl,...,0,0,0,0,0,1,2008,WD,Normal,118000


In [12]:
# drop rows with >= 2 nans
df_droprows = df_dropcol.dropna(thresh=df_dropcol.shape[1]-2)

In [13]:
df_droprows

,Id,MSSubClass,MSZoning,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,2,2008,WD,Normal,208500
1,2,20,RL,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,5,2007,WD,Normal,181500
2,3,60,RL,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,9,2008,WD,Normal,223500
3,4,70,RL,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,...,272,0,0,0,0,2,2006,WD,Abnorml,140000
4,5,60,RL,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,12,2008,WD,Normal,250000
5,6,50,RL,14115,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,0,320,0,0,700,10,2009,WD,Normal,143000
6,7,20,RL,10084,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,8,2007,WD,Normal,307000
7,8,60,RL,10382,Pave,IR1,Lvl,AllPub,Corner,Gtl,...,228,0,0,0,350,11,2009,WD,Normal,200000
8,9,50,RM,6120,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,205,0,0,0,0,4,2008,WD,Abnorml,129900
9,10,190,RL,7420,Pave,Reg,Lvl,AllPub,Corner,Gtl,...,0,0,0,0,0,1,2008,WD,Normal,118000


In [14]:
# fill the few remaining nans with the value on the next row
df_nonans = df_droprows.ffill(axis=0)

In [15]:
# check if any nan is left
df_nonans.isnull().values.any()

False

In [16]:
# convert into 1-hot encoding
features = pd.get_dummies(df_nonans)

In [17]:
features

,Id,MSSubClass,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,1,60,8450,7,5,2003,2003,196.0,706.0,0.0,...,0,0,0,1,0,0,0,0,1,0
1,2,20,9600,6,8,1976,1976,0.0,978.0,0.0,...,0,0,0,1,0,0,0,0,1,0
2,3,60,11250,7,5,2001,2002,162.0,486.0,0.0,...,0,0,0,1,0,0,0,0,1,0
3,4,70,9550,7,5,1915,1970,0.0,216.0,0.0,...,0,0,0,1,1,0,0,0,0,0
4,5,60,14260,8,5,2000,2000,350.0,655.0,0.0,...,0,0,0,1,0,0,0,0,1,0
5,6,50,14115,5,5,1993,1995,0.0,732.0,0.0,...,0,0,0,1,0,0,0,0,1,0
6,7,20,10084,8,5,2004,2005,186.0,1369.0,0.0,...,0,0,0,1,0,0,0,0,1,0
7,8,60,10382,7,6,1973,1973,240.0,859.0,32.0,...,0,0,0,1,0,0,0,0,1,0
8,9,50,6120,7,5,1931,1950,0.0,0.0,0.0,...,0,0,0,1,1,0,0,0,0,0
9,10,190,7420,5,6,1939,1950,0.0,851.0,0.0,...,0,0,0,1,0,0,0,0,1,0


In [18]:
# split into training and testing dataframes
train_features = features[features['SalePrice'] != -1]
test_features = features[features['SalePrice'] == -1]

In [19]:
train_features

,Id,MSSubClass,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,1,60,8450,7,5,2003,2003,196.0,706.0,0.0,...,0,0,0,1,0,0,0,0,1,0
1,2,20,9600,6,8,1976,1976,0.0,978.0,0.0,...,0,0,0,1,0,0,0,0,1,0
2,3,60,11250,7,5,2001,2002,162.0,486.0,0.0,...,0,0,0,1,0,0,0,0,1,0
3,4,70,9550,7,5,1915,1970,0.0,216.0,0.0,...,0,0,0,1,1,0,0,0,0,0
4,5,60,14260,8,5,2000,2000,350.0,655.0,0.0,...,0,0,0,1,0,0,0,0,1,0
5,6,50,14115,5,5,1993,1995,0.0,732.0,0.0,...,0,0,0,1,0,0,0,0,1,0
6,7,20,10084,8,5,2004,2005,186.0,1369.0,0.0,...,0,0,0,1,0,0,0,0,1,0
7,8,60,10382,7,6,1973,1973,240.0,859.0,32.0,...,0,0,0,1,0,0,0,0,1,0
8,9,50,6120,7,5,1931,1950,0.0,0.0,0.0,...,0,0,0,1,1,0,0,0,0,0
9,10,190,7420,5,6,1939,1950,0.0,851.0,0.0,...,0,0,0,1,0,0,0,0,1,0


In [20]:
test_features

,Id,MSSubClass,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,1461,20,11622,5,6,1961,1961,0.0,468.0,144.0,...,0,0,0,1,0,0,0,0,1,0
1,1462,20,14267,6,6,1958,1958,108.0,923.0,0.0,...,0,0,0,1,0,0,0,0,1,0
2,1463,60,13830,5,5,1997,1998,0.0,791.0,0.0,...,0,0,0,1,0,0,0,0,1,0
3,1464,60,9978,6,6,1998,1998,20.0,602.0,0.0,...,0,0,0,1,0,0,0,0,1,0
4,1465,120,5005,8,5,1992,1992,0.0,263.0,0.0,...,0,0,0,1,0,0,0,0,1,0
5,1466,60,10000,6,5,1993,1994,0.0,0.0,0.0,...,0,0,0,1,0,0,0,0,1,0
6,1467,20,7980,6,7,1992,2007,0.0,935.0,0.0,...,0,0,0,1,0,0,0,0,1,0
7,1468,60,8402,6,5,1998,1998,0.0,0.0,0.0,...,0,0,0,1,0,0,0,0,1,0
8,1469,20,10176,7,5,1990,1990,0.0,637.0,0.0,...,0,0,0,1,0,0,0,0,1,0
9,1470,20,8400,4,5,1970,1970,0.0,804.0,78.0,...,0,0,0,1,0,0,0,0,1,0


In [21]:
# create labels
y_train = np.array(train_features['SalePrice'])
y_train.shape

(1349,)

In [22]:
# drop goal feature
train_features_noprice = train_features.drop('SalePrice', axis = 1)
test_features_noprice = test_features.drop('SalePrice', axis = 1)

In [23]:
# create input features
x_train = np.array(train_features_noprice)
x_test = np.array(test_features_noprice)
print(x_train.shape, y_train.shape, x_test.shape)

(1349, 266) (1349,) (1346, 266)


In [24]:
# choose rf with 1000 trees (large enough, good training time)
rfr = RandomForestRegressor(n_estimators = 1000)

In [25]:
# train
rfr.fit(x_train, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=1000, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [26]:
# get predictions on training set
pred = rfr.predict(x_train)

In [27]:
# get RMSE
rms = sqrt(mean_squared_error(np.log(y_train), np.log(pred)))
print('RMSE =', rms)

RMSE = 0.05700122497811307


In [28]:
# get actual predition on testing set
pred_test = rfr.predict(x_test)

In [29]:
pred_test

array([123535.027, 154219.305, 183603.746, 182156.229, 201390.616,
       184467.888, 168892.36 , 176810.777, 180527.724, 123860.596,
       193975.01 ,  92055.626,  93218.263, 154307.702, 138007.35 ,
       394541.235, 253813.06 , 301180.646, 247728.648, 452198.242,
       316340.675, 210106.517, 178291.408, 177335.344, 171596.52 ,
       199482.938, 342381.414, 241592.25 , 214119.304, 198358.523,
       189622.089,  98447.75 , 175743.752, 309058.425, 298644.053,
       226489.523, 190438.845, 154950.202, 153767.974, 152146.515,
       174441.376, 167108.832, 300998.211, 228375.051, 215922.729,
       183569.148, 222210.603, 200566.462, 165026.424, 151828.436,
       152236.773, 173664.816, 153599.359, 187180.09 , 161511.618,
       155335.377, 132802.75 , 224101.482, 139247.382, 145789.15 ,
       160340.92 , 117177.276, 124831.959, 126238.48 , 131127.884,
       112407.886, 129484.121, 142971.699, 166696.684, 128214.324,
       149546.488, 116721.642, 161693.948, 115788.545,  80960.

In [30]:
# create prediction dataframe
data = {'Id': test_features_noprice['Id'], 'SalePrice': pred_test}
df = pd.DataFrame (data, columns = ['Id', 'SalePrice' ])

In [31]:
# save df to .csv
df.to_csv('pred_test.csv')